In [ ]:
""" Master File w/ hyperparameter sweeping across multiple architectures"""
"""
Restart kernel after running
Only need to run once
"""
!pip install scikit-learn matplotlib seaborn

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.utils.data as td
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset, Subset
import wandb

froot = './data/k562_samp_epft_norm_test_1.csv'
df = pd.read_csv(froot)

wandb.login()

print(df.head())

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hassett (elongation-net). Use `wandb login --relogin` to force relogin


  seqnames     start       end strand  ensembl_gene_id  score      ctcf  \
0       15  88623545  88623545      +  ENSG00000181026    0.0 -0.079992   
1       15  88623546  88623546      +  ENSG00000181026    0.0 -0.079942   
2       15  88623547  88623547      +  ENSG00000181026    0.0 -0.079893   
3       15  88623548  88623548      +  ENSG00000181026    0.0 -0.079844   
4       15  88623549  88623549      +  ENSG00000181026    0.0 -0.079796   

   h3k36me3   h3k4me1  h3k79me2   h3k9me1   h3k9me3  h4k20me1       sj5  \
0 -0.000099  0.348531  4.423451  0.446508 -0.168099  3.232475 -0.028916   
1  0.001638  0.352677  4.460072  0.453024 -0.169218  3.259194 -0.028916   
2  0.003360  0.356807  4.496664  0.459491 -0.170339  3.285849 -0.028916   
3  0.005065  0.360919  4.533223  0.465908 -0.171461  3.312435 -0.028916   
4  0.006754  0.365013  4.569743  0.472274 -0.172584  3.338952 -0.028916   

        sj3       dms  wgbs      rpts  lambda_alphaj      zeta  
0 -0.057178 -0.307549   0.0  0.24

In [ ]:
column_names = np.array(df.columns)
feature_names = column_names[6:-2]
num_features = len(feature_names)
print(feature_names)
num_samples = df.shape[0]

# process read counts per gene j, site i
X_ji = df['score'].values

# process GLM simulated elongation rates
Z_ji = df['zeta'].values

print("Number of Samples: " + str(num_samples))
print("Number of Features: " + str(num_features))

#Y_ji is a list of samples containing lists of their feature values
    # [   
    #   sample_1: [feat_1, feat_2,...,feat_n],
    #   sample_2: [feat_1, feat_2,...,feat_n],
    # ]

Y_ji = df.iloc[:, 6:-2].values
Y_ji_shape = Y_ji.shape
print(Y_ji.shape)

# read depth * initiation rate values per gene j
C_j = df['lambda_alphaj'].values

gene_ids = df['ensembl_gene_id'].values

cuda_available = torch.cuda.is_available()
print("CUDA (GPU support) is available:", cuda_available)
num_gpus = torch.cuda.device_count()
print("Number of GPUs available:", num_gpus)

In [ ]:
sweep_config = {
    'method': 'grid'
}
metric = {
    'name': 'valid_neural_net_loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'optimizer': {
        'values': ['adam']
    },
    'learn_rate': {
        'values': [1e-3]
    },
    'momentum': {
        'values': [0]#, 0.9]
    },
    'train_batch_size': {
        'values': [16000000]
    },
    'train_use_sliding_window': {
        'values': [False]#, True]
    },
    'train_window_size': {
        'values': [500]
    },
    'train_stride': {
        'values': [500]
    },
    'val_batch_size': {
        'values': [16000000]
    },
    'val_use_sliding_window': {
        'values': [False]#, True]
    },
    'val_window_size': {
        'values': [500]
    },
    'val_stride': {
        'values': [500]
    },
    'model_type': {
        'values': ['linear', 'lstm']
    },
    'bidirectional': {
        'values': [False]#, True]
    },
    'hidden_layer_size': {
        'values': [100]#, 50, 200, 500]
    },
    'num_layers': {
        'values': [1]#, 2]
    },
    'weight_init': {
        'values': [None]#, 'zero']
    }
}

parameters_dict.update({
    'epochs': {
        'value': 20}
    })

sweep_config['parameters'] = parameters_dict

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="elongation-net")

In [ ]:
class GeneDataset(Dataset):
    def __init__(self, grouped_data, use_sliding_window=False, window_size=None, stride=None):
        self.grouped_data = grouped_data
        self.use_sliding_window = use_sliding_window
        self.window_size = window_size
        self.stride = stride
        self.segments = []

        if self.use_sliding_window and window_size is not None and stride is not None:
            self._create_segments()
        else:
            self._prepare_full_genes()
        
    def _create_segments(self):
        for gene_id, group in self.grouped_data:
            gene_length = len(group)
            for start_idx in range(0, gene_length - self.window_size + 1, self.stride):
                end_idx = start_idx + self.window_size
                segment = group.iloc[start_idx:end_idx]
                self.segments.append((gene_id, segment))
    
    def _prepare_full_genes(self):
        for gene_id, group in self.grouped_data:
            self.segments.append((gene_id, group))

    def __len__(self):
        return len(self.segments)

    def __getitem__(self, idx):
        gene_id, segment = self.segments[idx]
        
        y_ji_array = np.array(segment['Y_ji'].tolist()).reshape(-1, 12)
        y_ji_tensor = torch.tensor(y_ji_array, dtype=torch.float64)
        
        data = segment.drop(columns=['GeneId', 'dataset', 'Y_ji'])
        tensor_data = torch.tensor(data.values, dtype=torch.float64)
        
        result = {
            'GeneId': gene_id,
            'Y_ji': y_ji_tensor,
            'gene_length': len(segment)
        }
        for col in data.columns:
            result[col] = tensor_data[:, data.columns.get_loc(col)]

        return result

In [ ]:
from sklearn.model_selection import train_test_split

data = pd.DataFrame({
    'GeneId': gene_ids,
    'Y_ji': [row for row in Y_ji],
    'X_ji': X_ji,
    'C_j': C_j,
    'Z_ji': Z_ji
})

grouped = data.groupby('GeneId')

# split by gene into train, val, test sets
train_idx, temp_idx = train_test_split(list(grouped.groups.keys()), test_size=0.2, random_state=42)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, random_state=42)

# create dictionary mapping each gene id to its assigned train, val, test dataset labels
dataset_mapping = {gene_id: 'train' for gene_id in train_idx}
dataset_mapping.update({gene_id: 'val' for gene_id in val_idx})
dataset_mapping.update({gene_id: 'test' for gene_id in test_idx})

# filter rows based on assigned dataset field
data['dataset'] = data['GeneId'].map(dataset_mapping)
train_data = data[data['dataset'] == 'train']
valid_data = data[data['dataset'] == 'val']
test_data = data[data['dataset'] == 'test']


print("train data size: " + str(len(train_data)))
print("val data size: " + str(len(valid_data)))
print("test data size: " + str(len(test_data)))

train_data = train_data.groupby('GeneId')
valid_data = valid_data.groupby('GeneId')
test_data = test_data.groupby('GeneId')
print("train # genes: " + str(len(train_data)))
print("val # genes: " + str(len(valid_data)))
print("test # genes: " + str(len(test_data)))

In [ ]:
def build_dataset(train_data, batch_size, use_sliding_window=False, window_size=None, stride=None):
    dataset = GeneDataset(train_data, use_sliding_window, window_size, stride)
    loader = DataLoader(dataset, batch_size=batch_size, num_workers=7, shuffle=False, pin_memory=True)
    return loader

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def build_model(model_type, num_layers, hidden_layer_size, bidirectional, weight_init):
    
    class LSTMModel(nn.Module):
        def __init__(self, input_size, hidden_layer_size, output_size, num_layers, bidirectional):
            super(LSTMModel, self).__init__()
            self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers, bidirectional=bidirectional, batch_first=True)
            self.bidirectional_linear = nn.Linear(2 * hidden_layer_size, output_size)
            self.linear = nn.Linear(hidden_layer_size, output_size)

        def forward(self, x):
            x, _ = self.lstm(x)
            if self.bidirectional:
                x = self.bidirectional_linear(x)
            else:
                x = self.linear(x)
            return x
    
    if model_type == 'lstm':
        model = LSTMModel(num_features, hidden_layer_size, 1, num_layers, bidirectional)
    elif model_type == 'linear':
        model = nn.Linear(num_features, 1, bias=False)
    
    if cuda_available:
        if num_gpus > 1:
            print("Using", num_gpus, "GPUs")
            model = torch.nn.DataParallel(model)
        model = model.to('cuda')

    print(model)

    first_param_device = next(model.parameters()).device
    print("Model is on device:", first_param_device)
    
    # expected weights are close to 0 which is why 0 initializing weights converges much quicker
    if weight_init == 'zero':
        with torch.no_grad():
            for param in model.parameters():
                param.zero_()
    
    model.double()

    return model.to(device)

In [ ]:
def build_optimizer(network, optimizer, learning_rate, momentum):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=momentum)
        
    # Adam optimizer adapts the learning rate for each parameter individually
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer

In [ ]:
def valid_epoch(model, loader, loss_fn):
    model.eval()
    total_neural_net_loss = 0
    total_glm_loss = 0
    neural_net_zeta = []
    glm_zeta = []
    with torch.no_grad():
        for idx, batch in enumerate(loader):
            Y_ji_batch = batch['Y_ji'].to(device)
            X_ji_batch = batch['X_ji'].to(device)
            C_j_batch = batch['C_j'].to(device)
            Z_ji_batch = batch['Z_ji'].to(device)
            lengths = batch['gene_length'].to(device)
            
            outputs = model(Y_ji_batch)
            neural_net_loss = loss_fn(X_ji_batch, C_j_batch, outputs.squeeze(2), lengths)
            glm_loss = loss_fn(X_ji_batch, C_j_batch, Z_ji_batch, lengths)

            total_neural_net_loss +=  neural_net_loss.item()
            total_glm_loss += glm_loss.item()
            
            # store all predictions in list
            neural_net_zeta.append(torch.exp(outputs.cpu()[0]))
            glm_zeta.append(batch['Z_ji'][0])
    
    # calculate average loss across all batches
    avg_neural_net_loss = total_neural_net_loss / len(loader)
    avg_glm_loss = total_glm_loss / len(loader)
    
    neural_net_zeta = torch.cat(neural_net_zeta, dim=0)
    glm_zeta = torch.cat(glm_zeta, dim=0)
    
    return avg_neural_net_loss, avg_glm_loss, neural_net_zeta, glm_zeta

In [ ]:
def train_epoch(model, loader, optimizer, loss_fn):
    model.train()
    total_loss = 0
    for idx, batch in enumerate(loader):
        optimizer.zero_grad()
        Y_ji_batch = batch['Y_ji'].to(device)
        X_ji_batch = batch['X_ji'].to(device)
        C_j_batch = batch['C_j'].to(device)
        lengths = batch['gene_length'].to(device)
        
        outputs = model(Y_ji_batch)
        loss = loss_fn(X_ji_batch, C_j_batch, outputs.squeeze(2), lengths)
        loss.backward()
        optimizer.step()
        
        # calculate average loss across all batches
        total_loss += loss.item()
    avg_train_loss = total_loss / len(loader)
    
    return avg_train_loss

In [ ]:
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, X_ji, C_j, rho_ji, lengths):
        C_j_value = C_j[0]
        loss = X_ji * rho_ji + C_j_value * torch.exp(-rho_ji) - X_ji * torch.log(C_j_value)
        
        # normalize loss by sequence length
        loss_sum = loss.sum(dim=1)
        normalized_loss = loss_sum / lengths.float()
        
        # calculate average loss within each batch
        return (normalized_loss).mean()

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
increase_cut=1
learning_rate_decreased = False

def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        model = build_model(config.model_type, config.num_layers, config.hidden_layer_size, 
                            config.bidirectional, config.weight_init)
        train_loader = build_dataset(train_data, config.train_batch_size, config.train_use_sliding_window, 
                                     config.train_window_size, config.train_stride)
        valid_loader = build_dataset(valid_data, config.val_batch_size, config.val_use_sliding_window, 
                                     config.val_window_size, config.val_stride)
        optimizer = build_optimizer(model, config.optimizer, config.learn_rate, config.momentum)
        
        loss_fn = CustomLoss()
        loss_neural_net_train = [0] * epochs
        loss_neural_net_valid = [0] * epochs
        loss_glm_valid = [0] * epochs
        
        # scheduler to reduce learning rate by half when new validation loss > old validation loss
        old_neural_net_valid_loss = float('inf')
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1, verbose=True)

        for epoch in range(config.epochs):
            print(f'Epoch {epoch+1}')
            
            train_loss = train_epoch(model, train_loader, optimizer, loss_fn)
            loss_neural_net_train[epoch] = train_loss
            print("train loss: "+ str(train_loss))
            
            valid_neural_net_loss, valid_glm_loss, neural_net_zeta, glm_zeta = valid_epoch(model, valid_loader, loss_fn)
            loss_neural_net_valid[epoch] = valid_neural_net_loss
            loss_glm_valid[epoch] = valid_glm_loss
            print("valid neural net loss: "+ str(valid_neural_net_loss))
            print("valid glm loss: "+ str(valid_glm_loss))
            
            # calculate metrics
            mae = F.l1_loss(net_zeta.squeeze(), glm_zeta)
            mse = F.mse_loss(net_zeta.squeeze(), glm_zeta)
            correlation_coefficient = np.corrcoef(glm_zeta, net_zeta.squeeze())[0, 1]
            print("Correlation Coefficient:", correlation_coefficient)
            print(f"Mean Absolute Error: {mae.item():.4f}")
            print(f"Mean Squared Error: {mse.item():.4f}")
            
            
            wandb.log({"epoch": epoch, "train_loss": train_loss, "valid_neural_net_loss": valid_neural_net_loss,
                       "valid_glm_loss": valid_glm_loss, "corr_coeff": correlation_coefficient, "mae": mae.item(), 
                       "mse": mse.item()})
            
            # early stopping if loss is not improving after reducing learning rate
            if learning_rate_decreased and valid_neural_net_loss - old_neural_net_valid_loss < increase_cut:
                break
                
            # reduce learning rate if new loss > old loss
            learning_rate_decreased = False
            if valid_neural_net_loss > old_neural_net_valid_loss:
                optimizer.param_groups[0]['lr'] *= 0.5
                print(f"Reduced learning rate to {optimizer.param_groups[0]['lr']}")
                learning_rate_decreased=True
            old_train_loss = train_loss
            scheduler.step(train_loss)
            
        return model

In [ ]:
wandb.agent(sweep_id, train)